In [55]:
#Import Libraries
import pygame
import random
import math
import time

In [56]:
#Define Circle class
class Circle:
    def __init__(self, x, y, radius, color):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.speed = random.uniform(0.5, 1.7)
        self.angle = random.uniform(0, 2 * math.pi)
    
    def move(self):
        self.x += self.speed * math.cos(self.angle)
        self.y += self.speed * math.sin(self.angle)
        
        # Wall collision
        if self.x + self.radius > SCREEN_WIDTH-10 or self.x - self.radius < 0:
            self.angle = math.pi - self.angle
        if self.y + self.radius > SCREEN_HEIGHT-10 or self.y - self.radius < 0:
            self.angle = -self.angle
    
    def draw(self):
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), self.radius)

In [57]:
# Movement logic for basketball to follow a player
def update_basketball(player, x_change, y_change):
    basketball.x = player.x+x_change
    basketball.y = player.y+y_change
    basketball.speed = player.speed
    basketball.angle = player.angle

In [58]:
# Simulate the basketball being passed
def move_basketball_to_player(basketball, target, players, exclusion_list, speed):
    
    adjusted_movement = adjust_path_if_needed(basketball, target, players, exclusion_list)
    basketball.x += adjusted_movement.x
    basketball.y += adjusted_movement.y

    # Check if reached target -> Distance between player and basketball.
    ### NOTE: AFTER CODING, SWITCH IF CONDITIONS BASED ON LARGER VALUE. Replace bottom value based on area overlap
    if pygame.math.Vector2(target.x - basketball.x, target.y - basketball.y).length() <= speed:
        if ((abs(basketball.y - target.y) <= 3) and (abs(basketball.x - target.x) <= 3)):
            return True
    else:
        return False

In [59]:
# Adjust basketball pass path

def adjust_path_if_needed(basketball, target, players, exclusion_list):

    # Calculate the direct path vector
    dx, dy = target.x - basketball.x, target.y - basketball.y
    path_vector = pygame.math.Vector2(dx, dy)
    
    for player in players:
        if player not in exclusion_list:
            # Calculate vector from basketball to player
            to_player_vector = pygame.math.Vector2(player.x - basketball.x, player.y - basketball.y)
            
            # Check if the player is close to the direct path
            distance_to_path = path_vector.cross(to_player_vector) / path_vector.length()
            
            if abs(distance_to_path) < player.radius + basketball.radius:
                # Adjust the path to avoid the player
                avoidance_vector = path_vector.normalize().rotate(10)
                return avoidance_vector * MOVE_SPEED
    
    # If no adjustment needed, return the original path vector
    return path_vector.normalize() * MOVE_SPEED


In [60]:
#Check for unwanted overlap during passes

def check_for_unwanted_overlap(basketball, players, exclude_players):
    for player in players:
        if (player not in exclude_players) and (circles_overlap(basketball, player, 0)):
            return True
    return False


In [61]:
# Circle placement

def is_valid_placement(new_circle, existing_circles):
    for circle in existing_circles:
        dx = new_circle.x - circle.x
        dy = new_circle.y - circle.y
        distance = math.sqrt(dx**2 + dy**2)
        # Calculate the minimum allowed distance between circle centers
        min_allowed_distance = circle.radius + new_circle.radius - 0.2 * min(circle.radius, new_circle.radius)
        if distance < min_allowed_distance:
            return False
    return True


def place_circle_with_constraints(existing_circles, radius, color):
    attempts = 0
    while attempts < 1000:  # Limit attempts to prevent infinite loop
        new_circle = Circle(random.randint(radius, SCREEN_WIDTH - radius),
                            random.randint(radius, SCREEN_HEIGHT - radius),
                            radius, color)
        if is_valid_placement(new_circle, existing_circles):
            return new_circle
        attempts += 1
    raise Exception("Failed to place a new player without exceeding overlap threshold.")

In [62]:
# Colliding of circles

def circles_overlap(circle1, circle2, minimum_overlap_percentage):
    # Calculate the distance between the centers of the two circles
    distance_centers = math.sqrt((circle1.x - circle2.x) ** 2 + (circle1.y - circle2.y) ** 2)
    
    # Calculate the sum of the radii
    sum_of_radii = circle1.radius + circle2.radius
    
    # Overlap Percentage
    adjusted_distance_for_overlap = sum_of_radii * (1 - minimum_overlap_percentage)

    return distance_centers <= adjusted_distance_for_overlap

In [63]:
# Import Basketball Court image

#Displayed Screen Dimensions
SCREEN_WIDTH = 500
SCREEN_HEIGHT = 500

#Load and Scale image
image_location = '/Users/abhishekramesh/Library/Mobile Documents/com~apple~CloudDocs/NBA Analysis Project/NBA Court.jpg'

background_image = pygame.image.load(image_location)
background_image = pygame.transform.scale(background_image, (SCREEN_WIDTH, SCREEN_HEIGHT))

In [64]:
# Simulation Parameters

# Simulation Constants
NUM_PLAYERS = 10
PLAYER_RADIUS = 20
BALL_RADIUS = 10
COLOR_BLUE = (0, 0, 255)
COLOR_RED = (255, 0, 0)
COLOR_ORANGE = (255, 165, 0)
COLOR_WHITE = (255, 255, 255)
FPS = 60
MOVE_SPEED = np.random.normal(5, 2/3) #Speed at which basketball moves to the next player - Normal Distribution (mu = 5, sigma = 4/6)

# Simulation Variables
clock = pygame.time.Clock()
pass_timer = -1
pass_interval = 0.1 #random.uniform(0.5, 5)  #Pass between players occur every 0.1 seconds
reached_player = True
running = True
basketball_relative_x = 5 #X-axis displacemnt of the basketball compared to the player
basketball_relative_y = 5 #Y-axis displacemnt of the basketball compared to the player
minimum_overlap_percentage = 0.4
first_overlap = False #Calculate the basketball's relative x, y coordinate to the circle 

# Initialize Pygame
pygame.init()

# Create screen
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("NBA Game Simulation")

# Initialize players and basketball
players = []
for i in range(NUM_PLAYERS):
    color = COLOR_BLUE if i < NUM_PLAYERS // 2 else COLOR_RED
    player = place_circle_with_constraints(players, PLAYER_RADIUS, color)
    players.append(player)

# Choose a random blue player to place the basketball with
blue_players = [player for player in players if player.color == COLOR_BLUE]
current_player = random.choice(blue_players)

basketball = Circle(
    current_player.x - basketball_relative_x, 
    current_player.y - basketball_relative_y, 
    BALL_RADIUS, COLOR_ORANGE
)

In [ ]:
#Simulate the game

while running:
    
    #Create screen with basketball court as the background
    screen.blit(background_image, (0,0))

    #Stop simulation
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Update and draw players
    for player in players:
        player.move()
        player.draw()

    if not reached_player:
        # Move basketball towards the current player
        
        # Create a new list excluding the current player and randomly choose a player to pass to
        blue_players_excluding_current = [player for player in blue_players if player != current_player]
        new_random_player = random.choice(blue_players_excluding_current)
        
        pass_players = [current_player, new_random_player] #2 players the ball is being passed between
        reached_player = move_basketball_to_player(basketball, current_player, players, pass_players, MOVE_SPEED)
        
        #Calculate the distance between the basketball and the receiving player
        if first_overlap == False:
            if circles_overlap(basketball, current_player, minimum_overlap_percentage):
                #Calculate x, y relative to player
                basketball_relative_x = abs(current_player.x - basketball.x) #X-axis displacemnt of the basketball compared to the player
                basketball_relative_y = abs(current_player.y - basketball.y) #Y-axis displacemnt of the basketball compared to the player
                first_overlap = True
    else:  
        # Start timer once basketball reaches player
        update_basketball(current_player, basketball_relative_x, basketball_relative_y)
        if pass_timer < 0:
            pass_timer = 0  # Activate timer
        pass_timer += clock.get_time() / 1000.0  # Convert milliseconds to seconds
        
        if pass_timer >= pass_interval:
            # Time to pass the basketball to the next blue player
            current_index = blue_players.index(current_player) + 1
            if current_index >= len(blue_players):
                current_index = 0
            current_player = blue_players[current_index]
            pass_timer = -1  # Reset timer to deactivate
            first_overlap = False
            reached_player = False
    
    basketball.draw()  # Draw the basketball
    
    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()


In [ ]:
#Things to Implement
'''
- Save videos of passes
--> Create snips of videos when basketball moves between 2 players

- Players move more naturally
--> Player speed changes ()

- Base Basketball Actions
//Steal
- If opposing player comes in contact with basketball
--> Opposing player takes control of the ball
--> Passing team = opposing team

//Shooting
- Shooting basketball into basket
--> After basketball passed 3 times, player shoot basketball moves into basket circle

- Made shot
--> Basketball falls straight through

- Missed shot
--> Basketball moves randomly within a X radius (not behind backboard)

//Rebound
- When shot misses, player goes for the rebound
--> Players closeset to the ball moves to the ball
--> First player to reach the ball, gets the ball


- Match play by play to simulation
--> Get NBA play by play data
--> Match play

'''